# Домашнее задание № 7

## по дисциплине: Теория вероятностей и математическая статистика

## Тема: Линейная регрессия. Двухвыборочный t-тест. A/B-тестирование

### выполнил: Евгений Кириллов

__Задача 1__

Дана матрица объект-признак
```
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]
```
и значения целевой переменной
```
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
```

Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

__Ответ__: 0.76342462

In [1]:
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]

y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]

In [2]:
import numpy as np

In [4]:
X = np.array(X)
y = np.array(y)

Итак, нашу задачу можно решить, построив для каждой пары признаков модель линейной
регрессии на этих признаках, и посчитав её коэффициент детерминации.

Коэффициенты линейной регрессии можно посчитать по формуле: $$b = \left( X^\top X \right)^{-1}
X^\top y$$

Здесь $X$ — «расширенная» матрица, т.е. первый столбец этой матрицы соответствует значению $1$ при коэффициенте $b_0$.

После этого для вычисления коэффициента детерминации нам понадобятся предсказанные
моделью значения: $$z = X \cdot b$$

По ним вычисляется массив ошибок модели: $$E = y - z$$

Теперь коэффициент детерминации равен: $$R^2 = 1 - \dfrac{SS_{res}^2}{SS_y^2}$$

Обернём всё это в единую функцию:

In [5]:
def sum_of_squares(samples: np.ndarray) -> float:
    """Сумма квадратов отклонений.
    """
    return ((samples - samples.mean()) ** 2).sum()

In [6]:
def get_determination_coefficient(x: np.ndarray, y: np.ndarray) -> float:
    """Эта функция строит по матрице объект-признак `x` и массиву значений
    целевой переменной `y` модель линейной регрессии и возвращает её
    коэффициент детерминации.
    """
    ones = np.ones((x.shape[0], 1))
    x = np.hstack([ones, x])
    xtx = x.T.dot(x)
    xtx_inv = np.linalg.inv(xtx)
    b = xtx_inv.dot(x.T).dot(y)
    z = x.dot(b)
    E = y - z
    
    return 1 - sum_of_squares(E) / sum_of_squares(y)

Итак, переберём все пары признаков и посчитаем для каждой такой пары коэффициент
детерминации соответствующей модели.

In [7]:
from itertools import combinations

In [8]:
for i, j in combinations(range(X.shape[1]), 2):
    r = get_determination_coefficient(X[:, [i, j]], y)
    print(f'{i} {j} {r}')

0 1 0.18113594742585204
0 2 0.7634246238793152
0 3 0.45329667831440745
1 2 0.5479482734039012
1 3 0.6062055761129932
2 3 0.6224419876505322


__Задача 2__

Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.

__Ответ__: 1

Видим, что использование первого и третьего признаков даёт наилучшую зависимость.
Проверим статистическую значимость этой зависимости. Это можно сделать с помощью
распределения Фишера. Используем статистику: $$F = \dfrac{R^2 / m}{(1 - R^2) / (n - m - 1)},$$
где $m$ — число факторов модели. В нашем случае $m = 2$. Такая статистика имеет
распределение Фишера с параметрами $k_1 = m$, $k_2 = n - m - 1$.

In [9]:
k1 = 2
k2 = X.shape[0] - k1 - 1
R = get_determination_coefficient(X[:, [0, 2]], y)
F = (R / k1) / ((1 - R) / k2)
F

11.29443912292265

Для уровня значимости $\alpha = 0.05$ найдём нужный квантиль (напомним, что критическая
область у распределения Фишера правосторонняя):

In [10]:
from scipy import stats

In [11]:
alpha = 0.05
t = stats.f.ppf(1 - alpha, k1, k2)
t

4.73741412777588

Итак, критическая область имеет вид: $$\Omega_\alpha = \left( 4.737, \infty \right)$$
Значение статистики попало в критическую область, значит, уравнение регрессии признаётся
статистически значимым.

__Задача 3__

Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_. Реализуйте двухвыборочный t-тест. В качестве выборок здесь можно взять наборы меток совершения покупки (0 или 1) каждым посетителем.

__Ответ__: 0

Итак, запишем выборки:

In [13]:
x1 = np.zeros(2509)
x1[np.arange(77)] = 1
x2 = np.zeros(1465)
x2[np.arange(60)] = 1

Реализуем двухвыборочный t-тест.

In [14]:
n1 = x1.size
n2 = x2.size
s1 = x1.std(ddof=1)
s2 = x2.std(ddof=1)

Посчитаем $\sigma_\Delta$: $$\sigma_\Delta = \sqrt{\dfrac{\sigma_{X_1}^2}{n_1} +
\dfrac{\sigma_{X_2}^2}{n_2}}$$

In [15]:
s_delta = np.sqrt(s1 ** 2 / n1 + s2 ** 2 / n2)
s_delta

0.006220171278295827

Значение статистики: $$t = \dfrac{\overline{X_1} - \overline{X_2}}{\sigma_\Delta}$$

In [16]:
t = (x1.mean() - x2.mean()) / s_delta
t

-1.6504551408398205

Посчитаем число степеней свободы: $$df = \dfrac{\left( \dfrac{\sigma_{X_1}^2}{n_1} +
\dfrac{\sigma_{X_2}^2}{n_2} \right)^2}{\dfrac{\left( \sigma_{X_1}^2 / n_1 \right)^2}{n_1 - 1} +
\dfrac{\left( \sigma_{X_2}^2 / n_2 \right)^2}{n_2 - 1}}$$

In [17]:
df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 / ((s1 ** 2 / n1) ** 2 / (n1 - 1) +
(s2 ** 2 / n2) ** 2 / (n2 - 1))
df

2732.8025644352133

Возьмём уровень значимости $\alpha = 0.05$. Посчитаем квантили распределения Стьюдента:

In [18]:
alpha = 0.05
t1 = stats.t.ppf(alpha / 2, df=df)
t2 = stats.t.ppf(1 - alpha / 2, df=df)
t1, t2

(-1.9608324352746576, 1.9608324352746571)

Итак, критическая область выглядит следующим образом: $$\Omega_\alpha = (-\infty, -1.96) \cup
(1.96, \infty)$$
Значение статистики в эту область не попало, поэтому заключаем, что между конверсией на
страницах A и B значимого отличия нет.